In [ ]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

In [ ]:
!ls ../data/hw2

In [ ]:
# считаем данные в соответствующие датафреймы
FLD = "/home/nur/projects/analysis/dynamic_price/data/hw2"
train_main_df = pd.read_csv(os.path.join(FLD, 'HW_train_main_data.csv'))
train_additional_df = pd.read_csv(os.path.join(FLD, 'HW_train_additional_data.csv'))
train_main_df['timestamp'] = pd.to_datetime(train_main_df['timestamp'])

In [ ]:
test_main_df = pd.read_csv(os.path.join(FLD, 'HW_test_main_data.csv'))
test_additional_df = pd.read_csv(os.path.join(FLD, 'HW_test_additional_data.csv'))
test_main_df['timestamp'] = pd.to_datetime(test_main_df['timestamp'])

In [ ]:
# посмотрим на колонки, информацию о пустых значениях и типах данных
train_main_df.info()

## EDA: Exploratory data analysis

In [ ]:
train_main_df.describe()

#### 1. Найти id топ-10 самых дорогих квартир из датасета.

In [ ]:
res = dict()
r = train_main_df.nlargest(5, columns = 'full_sq')[['id', 'full_sq']]
res[3] = r.id.values.tolist()
res[3]

#### 2. Построить зависимость средней стоимости квартиры от года и месяца продаж.

In [ ]:
train_main_df.timestamp = pd.to_datetime(train_main_df.timestamp)
train_main_df['year'] = train_main_df.timestamp.dt.year
train_main_df['month'] = train_main_df.timestamp.dt.month
train_main_df['year_month'] = train_main_df['year'].astype(str) + '_' + train_main_df['month'].astype(str)
train_main_df.head(1)

In [ ]:
plt.close("all")
train_main_df.groupby('year_month').mean().sort_values(by=['year', 'month']).plot(y="price", figsize=(10,5))

#### 3. Для каждой пары месяц-год найти индексы (не id) самых дорогих квартир.

In [ ]:
res[2] = train_main_df[['year_month', 'price']].groupby('year_month').idxmax()['price'].astype(int).values.tolist()
res[2][:5]

#### 4. Построить boxplot для цены для пар месяц-год. 

In [ ]:
plt.figure(figsize=(13, 6))
sns.boxplot(x='year_month', y='price', data=train_main_df.sort_values(by=['year', 'month']))
plt.ylabel('price', fontsize=12)
plt.xlabel('year_month', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

#### 5. Найти id топ-5 самых больших квартир.  

In [ ]:
r = train_main_df.nlargest(10, columns = 'price')[['id', 'price']]
res[1] = r.id.values.tolist()
res[1]

#### 6. Посчитать количество пропусков в life_sq.  

In [ ]:
res[4] = train_main_df.life_sq.isnull().sum(axis=0)
res[4]

#### 7. Заполнить пропуски life_sq

In [ ]:
# как вариант можно построить KNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
clf = KNeighborsRegressor()
params = {'n_neighbors':[2,3,4,5,6,7,8,9,10], 'weights': ['uniform', 'distance']}
grcv = GridSearchCV(clf, params, n_jobs=-1)
grcv.fit(train_main_df[~train_main_df.life_sq.isnull()].full_sq.values.reshape(-1,1)
         , train_main_df[~train_main_df.life_sq.isnull()].life_sq.values.reshape(-1,1))

grcv.best_params_

#### 8. Сохранить коэффициенты корреляции Пирсона между (price, full_sq) и (price, life_sq без пропусков). 

In [ ]:
df = train_main_df[['price', 'full_sq', 'life_sq']]
df = df[~df.isnull()]
price_full_sq = df.corrwith(df.full_sq, axis=0, drop=False, method='pearson')[0]
price_life_sq = df.corrwith(df.life_sq, axis=0, drop=False, method='pearson')[0]
res[5] = [price_full_sq, price_life_sq]
res[5]

In [ ]:
fpath_out = 'part1.csv'
with open(fpath_out, 'w') as f:
    for i in range(1, 6):
        if type(res[i]) == list:
            f.write(', '.join(map(str, res[i])))
        else:
            f.write(str(res[i]))
        f.write('\n')

In [ ]:
!cat part1.csv

## Continue Analysis & Data input

In [ ]:
def empty_analysis(df):
    missing_df = df.isnull().sum(axis=0).reset_index()
    missing_df.columns = ['column_name', 'missing_count']
    missing_df = missing_df.loc[(missing_df['missing_count'] > 0), :]
    missing_df = missing_df.sort_values(by='missing_count')
    ind = range(missing_df.shape[0])

    fig, ax = plt.subplots(figsize=(10,5))
    rects = ax.barh(ind, missing_df['missing_count'], color="blue")
    ax.set_yticks(ind)
    ax.set_yticklabels(missing_df.column_name.values, rotation='horizontal')
    ax.set_xlabel("Count of missing values")
    ax.set_title("Number of missing values in each column")
    plt.show()

empty_analysis(train_main_df)

In [ ]:
whole_train_df = train_main_df.merge(train_additional_df, how='left', on='id')
whole_test_df = test_main_df.merge(test_additional_df, how='left', on='id')

In [ ]:
def barplot(colname, th = None):
    plt.figure(figsize=(12, 5))
    if th:
        df = whole_train_df[whole_train_df[colname] >= th]
    else:
        df = whole_train_df
    sns.boxplot(x=colname, y='price', data=df)
    plt.ylabel('price', fontsize=12)
    plt.xlabel(colname, fontsize=12)
    plt.xticks(rotation='vertical')
    plt.show()

barplot('build_year', th = 2000)

# Data preparation

In [ ]:
# def fillna_const(df):
#     consts = {
#         'kitch_sq':-20,
#               'hospital_beds_raion':0,
#               'num_room':-20,
# #               'materials':1
#               }
#     df.fillna(consts, inplace=True)

# fillna_const(whole_train_df)
# fillna_const(whole_test_df)

In [ ]:
# import random
# def build_year_col(df):
#     idx = (df.build_year < 1800) | (df.build_year > 2015)
#     df.loc[idx, 'build_year'] = df[idx].max_floor.apply(lambda x: random.randint(1970, 2005) if x < 12 else random.randint(1998, 2007))
    
# build_year_col(whole_train_df)
# build_year_col(whole_test_df)

In [ ]:
# добавим дополнительные столбцы на основе имеющейся даты
def date_newcol(df):
    df['year'] = df.timestamp.dt.year

    df['month'] = df.timestamp.dt.month

    df['week_of_year'] = df.timestamp.dt.isocalendar().week.astype(int)

    df['day_of_week'] = df.timestamp.dt.weekday

    df['timestamp_int'] = df.timestamp.astype(int)

    df['year_month'] = df['year'].astype(str) + '_' + df['month'].astype(str)
    
date_newcol(whole_train_df)
date_newcol(whole_test_df)

In [ ]:
# def max_floor_col(df):
#     idx = df.max_floor.isnull()
#     df.loc[idx, 'max_floor'] = df[idx].floor.apply(lambda x: max(x+1, 15))

# max_floor_col(whole_train_df)
# max_floor_col(whole_test_df)

In [ ]:
def bins_col(df):
    bins = [0, 30, 40, 50, 60, 70, 80, 90, 100, 200, 5326]
    df['full_sq_bins'] = np.searchsorted(bins, df.full_sq.values)    
    
    bins = [0, 6, 10, 17, 26, 33, 46]
    df['max_floor_bins'] = np.searchsorted(bins, df.max_floor.values)

    bins = [0, 1917, 1950, 1960, 1978, 1991, 2000, 2006, 2011]
    df['build_year_bins'] = np.searchsorted(bins, df.build_year.values)

bins_col(whole_train_df)
bins_col(whole_test_df)

In [ ]:
def area_update(df):
    idx = df.life_sq.isnull()
    # добавим разность между общей и жилой площадью квартиры
    df['pred_life_sq'] = np.NaN
    no_life_sq = grcv.predict(df[idx].full_sq.values.reshape(-1,1))
    df.loc[idx, 'pred_life_sq'] = list(no_life_sq.reshape(1,-1)[0])
    
    df['some_extra_sqr_2'] = df["full_sq"] - df.life_sq.combine_first(df.pred_life_sq)
    
#     df.loc[idx, 'life_sq'] = df[idx].full_sq - df[idx].kitch_sq
    df['some_extra_sqr'] = df["full_sq"] - df["life_sq"]
    
    assert df.some_extra_sqr_2.isnull().sum(axis=0) == 0
#     assert df.life_sq.isnull().sum(axis=0) == 0
#     assert df.kitch_sq.isnull().sum(axis=0) == 0
#     assert df.some_extra_sqr.isnull().sum(axis=0) == 0

area_update(whole_train_df)
area_update(whole_test_df)

In [ ]:
# заполним все пропуски константой
whole_train_df.fillna(-20, inplace=True)
whole_test_df.fillna(-20, inplace=True)

In [ ]:
def ratio_newcol(df):
    # вспомним, что цена сильно зависит от площади квартиры, на основе этих данных
    # добавим столбцы для отношения площадей
    sigma = 1e-8
    df["ratio_life_dash_full_sq"] = df["life_sq"] / (df["full_sq"] + sigma)
    df["ration_kitchen_dash_full_sq"] = df["kitch_sq"] / (df["full_sq"] + sigma)
    df["ration_extra_dash_full_sq"] = df["some_extra_sqr"] / (df["full_sq"] + sigma)
    df["floor_dash_max_floor"] = df["floor"] / (df["max_floor"] + sigma)
    df["avg_room_sq"] = df['full_sq'] / (df.num_room + sigma)

    # добавим воздраст здания
    df['age'] = df["build_year"] - df['year']

ratio_newcol(whole_train_df)
ratio_newcol(whole_test_df)

In [ ]:
from datetime import date
from currency_converter import CurrencyConverter

def add_currency_col(df):
    c = CurrencyConverter(fallback_on_missing_rate=True)
    df['usd_dash_rub'] = df.timestamp.apply(lambda x: c.convert(1, 'USD', 'RUB', date=x))
#     df['eur_dash_rub'] = df.timestamp.apply(lambda x: c.convert(1, 'EUR', 'RUB', date=x))
    
    dfg = df.groupby('year_month').usd_dash_rub.mean()
    d = dict(zip(dfg.index, dfg.values))
    df['usd_month'] = df.year_month.apply(d.get).astype(int)
    
#     dfg = df.groupby('year_month').eur_dash_rub.mean()
#     d = dict(zip(dfg.index, dfg.values))
#     df['eur_month'] = df.year_month.apply(d.get).astype(int)
    
add_currency_col(whole_train_df)
add_currency_col(whole_test_df)

In [ ]:
# Repair vals
whole_train_df.loc[whole_train_df['apartment condition'] == 33, 'apartment condition'] = 3
# Remove outliers full_sq
idx = (whole_train_df.full_sq_bins == 4) & (whole_train_df.price > 1e8)
whole_train_df.drop(whole_train_df[idx == True].index, inplace=True)

In [ ]:
whole_train_df.info()

In [ ]:
whole_train_df.life_sq.hist()

# Modeling

In [ ]:
def log_vals(df):
    df['full_sq'] = np.log2(df.full_sq)
#     df['population'] = np.log2(df.population)
#     df['office_num'] = np.log2(df.office_num)
#     df['life_sq'] = np.log2(df.life_sq)
    
    df.fillna(0, inplace=True)
    df.replace([np.inf, -np.inf], 0, inplace=True)
#     df['full_sq'] = df.full_sq.astype(int)
#     df['population'] = df.population.astype(int)
#     df['office_num'] = df.office_num.astype(int)
#     df['life_sq'] = df.life_sq.astype(int)
    

log_vals(whole_train_df)
log_vals(whole_test_df)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [ ]:
col_list = ['full_sq', 'life_sq', 'floor', 'max_floor',
       'material', 'build_year', 'num_room', 'kitch_sq', 'apartment condition',
       'sub_area', 'full_sq_bins', 'population', 'indust_part',
       'preschool_facilities', 'school_facilities', 'hospital_beds_raion',
       'healthcare_facilities', 'university_num', 'sport_objects_facilities',
       'additional_education_facilities', 'culture_objects_facilities',
       'shopping_centers_facilities', 'office_num', 'green_part', 'prom_part',
       'cafe_count', 'church_facilities', 'mosque', 'leisure_facilities',
       'year', 'month', 'week_of_year', 'day_of_week', 'timestamp_int',
       'ratio_life_dash_full_sq','ration_kitchen_dash_full_sq',
       'age', 'some_extra_sqr', 'some_extra_sqr_2',
#             'ration_extra_dash_full_sq',
            'floor_dash_max_floor',
            'avg_room_sq', 
            'build_year_bins', 
            'max_floor_bins',
#             'usd_month'
           ]

set(whole_train_df.columns) - set(col_list)

In [ ]:
assert not any(whole_train_df[col_list].isnull().sum(axis=0).values)

In [ ]:
# теперь выберем для валидации случайные записи, а не деление по времени
X_train, X_test, y_train, y_test = train_test_split(
    whole_train_df[col_list],
    whole_train_df.price, test_size=1425, random_state=42)
# 1425

In [ ]:
# xgb_params = {
#     'eta': 0.05,
#     'max_depth': 4,
#     'subsample': 0.7,
#     'colsample_bytree': 0.7,
#     'objective': 'reg:linear',
#     'eval_metric': 'rmse',
#     'min_child_weight':1,
#     'n_estimators': 100,
#     'silent': 1,
#     'seed':0
# }

xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'min_child_weight':1,
    'silent': 1,
    'seed':0
}

In [ ]:
xgb_train = xgb.DMatrix(X_train, y_train, feature_names = col_list, enable_categorical=True)
xgb_test = xgb.DMatrix(X_test, y_test, feature_names = col_list, enable_categorical=True)
evallist = [(xgb_test, 'eval'), (xgb_train, 'train')]

In [ ]:
model = xgb.train(params = xgb_params, 
                    dtrain = xgb_train, 
                    num_boost_round = 2000, 
                    evals = evallist, 
                    early_stopping_rounds = 10, 
                    verbose_eval = 10)

In [ ]:
xgb_train_final = xgb.DMatrix(whole_train_df[col_list], whole_train_df.price, feature_names = col_list, enable_categorical=True)
final_model = xgb.train(params = xgb_params, 
                    dtrain = xgb_train_final, 
                    num_boost_round = 880)

### Final predictions

In [ ]:
xgb_val = xgb.DMatrix(whole_test_df[col_list], feature_names = col_list, enable_categorical=True)
preds = final_model.predict(data=xgb_val)

In [ ]:
# id,predicted_price
# 10,11.323928429511037

fpath_out = 'prediction.csv'
with open(fpath_out, 'w') as f:
    f.write('id,predicted_price\n')
    for id_val, price in zip(whole_test_df.id.values, preds):
        f.write(f"{id_val},{price}\n")


In [ ]:
!wc -l prediction.csv

In [ ]:
!explorer.exe .


# Post analysis 

In [ ]:
from xgboost import plot_importance

In [ ]:
# full_sq - самая важная, при этом падение в важности заметное
# можно подумать над исправлением данного момента
plot_importance(model,max_num_features=20, height=0.9)

In [ ]:
whole_train_df['sub_area']

In [ ]:
# посмотрим на ошибки наших предсказаний

scores = pd.DataFrame(val_y)

In [ ]:
scores['predicted'] = model.predict(xgb_test)

In [ ]:
scores['error'] = scores.price - scores.predicted

In [ ]:
scores.head(2)

In [ ]:
scores['error'].describe()

In [ ]:
# зная примеры, на которых большие ошибки, можно пробовать тюнить модель

In [ ]:
idxs

In [ ]:
idxs = scores.nlargest(20, columns = 'error').index
whole_train_df[whole_train_df.index.isin(idxs)]